In [50]:
import cv2 # pip install opencv-python

In [51]:
import matplotlib.pyplot as plt # pip install matplotlib

get pretrained config file & frozen model from darknet github repo

In [52]:
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'

Set Model

In [53]:
model = cv2.dnn_DetectionModel(frozen_model, config_file)

Set Class Labels (coconames)

In [54]:
classLabels = [] # empty list of python
file_name = 'coconames.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    #classLabels.append(fpt.read())

In [55]:
print(classLabels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [56]:
print( len(classLabels) )

80


In [57]:
model.setInputSize(320, 320)
model.setInputScale(1.0/127.5) ## 255/2 = 127.5
model.setInputMean( (127.5, 127.5, 127.5) ) ## moblilenet => [-1, 1]
model.setInputSwapRB(True)
# it will perform automatically. We don't need to convert this each time

< cv2.dnn.Model 000001F133802B30>

# read an image

In [58]:

def objDetect():
    img_name = "img.jpeg"
    img = cv2.imread('uploads/'+img_name)
    plt.imshow( cv2.cvtColor(img, cv2.COLOR_BGR2RGB) ) # actual image

    try:
        # detect objects
        ClassIndex, confidece, bbox = model.detect(img, confThreshold=0.60)
        # print(ClassIndex)
    except Exception:
        if (len(ClassInd)<1):
            return
    
    font_scale = 2
    font = cv2.FONT_HERSHEY_PLAIN
    for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidece.flatten(), bbox):
        # cv2.rectangle( frame, (x,y), (x+w, y+h)), (255, 0, 0), 2)
        # cv2.putText( img, text, (text_offset_x, text_offset_y), font, fontSclae=font_scale, color(0,0,0), thickness=1 )
        cv2.rectangle(img, boxes, (255, 0, 0), 2) #rectangle color is blue
        cv2.putText(img, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+30), font, fontScale=font_scale, color=(0, 0, 255), thickness=2)
    
    cv2.imwrite('uploads/img_out.jpeg', img) # save boxed image (original color)
    
    outputImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # image boxed to cv2.COLOR_BGR2RGB. Otherwise imshow() shows degraded image.
    plt.imshow( outputImg ) # see original color
    
    


In [60]:
from flask import Flask, render_template, request, redirect, send_from_directory
import os
from werkzeug.utils import secure_filename
from werkzeug.exceptions import RequestEntityTooLarge

app = Flask(__name__, template_folder="templates/")
app.config['UPLOAD_DIRECTORY'] = 'uploads/'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB
app.config['ALLOWED_EXTENSIONS'] = ['.jpg', '.jpeg', '.png', '.gif']


@app.route('/')
def index():
    files = os.listdir(app.config['UPLOAD_DIRECTORY'])
    images = []
    for filename in files:
        extension = os.path.splitext(filename)[1].lower()
        if extension in app.config['ALLOWED_EXTENSIONS']:
            images.append(filename)
    return render_template('index.html', images=images)

@app.route('/upload', methods=['POST'])
def upload():
    try:
        file = request.files['file_name']
        extension = os.path.splitext(file.filename)[1].lower()

        if file and extension in app.config['ALLOWED_EXTENSIONS']:
            file.save(os.path.join(
                app.config['UPLOAD_DIRECTORY'],
                'img.jpeg'
            ))
            # call object detection function
            objDetect()  
        else:
            return 'File is not an allowed image type.'

    except RequestEntityTooLarge:
        return 'File is larger than the 16MB limit.'

    return redirect('/')

@app.route('/serve-image/<filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(app.config['UPLOAD_DIRECTORY'], filename)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
